In [3]:
import torch

print(f"PyTorch version: {torch.__version__}")
print(f"is CUDA available ? {torch.cuda.is_available()}")

if torch.cuda.is_available():
    print(f"Card name : {torch.cuda.get_device_name(0)}")

PyTorch version: 2.5.1+cu121
is CUDA available ? True
Card name : NVIDIA GeForce GTX 1650 with Max-Q Design


After installing Pytorch, we run a small verification to see if we have the right version of cuda and the one that uses GPU to train faster

In [4]:
import torch.nn as nn
import torch.nn.functional as F

class MarsCNN(nn.Module):
    def __init__(self):
        super(MarsCNN, self).__init__()
        # 1 input channel (gray), 16 output channels, 3x3 kernel
        self.conv1 = nn.Conv2d(1, 16, kernel_size=3, padding=1)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        
        # 16 input channels, 32 output channels, 3x3 kernel
        self.conv2 = nn.Conv2d(16, 32, kernel_size=3, padding=1)
        
        # After two 2x2 pools, 64x64 becomes 16x16
        # 32 channels * 16 * 16 = 8192 features
        self.fc1 = nn.Linear(32 * 16 * 16, 128)
        self.fc2 = nn.Linear(128, 24) # 24 Output classes

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 32 * 16 * 16) # Flatten the images
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

# Instantiate the model and move to GPU
model = MarsCNN().to('cuda') 
print(model)

MarsCNN(
  (conv1): Conv2d(1, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (fc1): Linear(in_features=8192, out_features=128, bias=True)
  (fc2): Linear(in_features=128, out_features=24, bias=True)
)


In [5]:
import os
print(os.listdir(r"C:\Users\msi\Desktop\TECH_UP\accelerated_network_SoC\data"))


['data_prepare.ipynb', 'test', 'train', 'validation']


In [6]:
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

# Transformations
transform = transforms.Compose([
    transforms.Grayscale(num_output_channels=1),
    transforms.ToTensor(), # Just 0.0 to 1.0, no normalization
])
class_names = sorted(os.listdir(r"C:\Users\msi\Desktop\TECH_UP\accelerated_network_SoC\data\train"))
class_to_idx = {name: i for i, name in enumerate(class_names)}

# Datasets
train_dataset = datasets.ImageFolder(
    root=r"C:\Users\msi\Desktop\TECH_UP\accelerated_network_SoC\data\train",
    transform=transform
)
master_mapping = train_dataset.class_to_idx
val_dataset = datasets.ImageFolder(
    root=r"C:\Users\msi\Desktop\TECH_UP\accelerated_network_SoC\data\validation",
    transform=transform
)
val_dataset.class_to_idx = class_to_idx
test_dataset = datasets.ImageFolder(
    root=r"C:\Users\msi\Desktop\TECH_UP\accelerated_network_SoC\data\test",
    transform=transform
)
test_dataset.class_to_idx = class_to_idx

# DataLoaders
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

print(f"Found {len(train_dataset)} images for training.")
print(f"Found {len(val_dataset)} images for validation.")
print(f"Found {len(test_dataset)} images for testing.")
print(f"Class mapping: {train_dataset.class_to_idx}")

print(f"✅ Mapping Synced: {len(master_mapping)} classes identified.")


Found 4673 images for training.
Found 990 images for validation.
Found 1028 images for testing.
Class mapping: {'apxs': 0, 'apxs cal target': 1, 'chemcam cal target': 2, 'chemin inlet open': 3, 'drill': 4, 'drill holes': 5, 'drt front': 6, 'drt side': 7, 'ground': 8, 'horizon': 9, 'inlet': 10, 'mahli': 11, 'mahli cal target': 12, 'mastcam': 13, 'mastcam cal target': 14, 'observation tray': 15, 'portion box': 16, 'portion tube': 17, 'portion tube opening': 18, 'rems uv sensor': 19, 'rover rear deck': 20, 'scoop': 21, 'turret': 22, 'wheel': 23}
✅ Mapping Synced: 24 classes identified.


In [7]:
import torch.optim as optim


# 1. Setup Device (using your GTX 1650)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Check exactly what the model sees
train_dataset = datasets.ImageFolder(root=r"C:\Users\msi\Desktop\TECH_UP\accelerated_network_SoC\data\train",transform=transform)
num_classes = len(train_dataset.classes)
print(f"Model will output {num_classes} classes")

# Update your model initialization
model = MarsCNN()
model.fc2 = nn.Linear(128, num_classes) # Forces match with folders
model = model.to(device)

# 4. Loss Function and Optimizer
# CrossEntropyLoss is ideal for multi-class Martian terrain.
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001) # Adam is fast on GPUs.


Model will output 24 classes


In [8]:
num_epochs = 10

for epoch in range(num_epochs):
    model.train() # Set to training mode
    running_loss = 0.0
    
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        
        optimizer.zero_grad() # Reset the gradients
        
        # Forward Pass: Predict
        outputs = model(images)
        loss = criterion(outputs, labels)
        
        # Backward Pass: Learn
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
    
    # --- Validation Phase ---
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad(): # No need to calculate gradients for checking
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
            
    model.eval()
    with torch.no_grad():
        images, labels = next(iter(val_loader))
        outputs = model(images.to(device))
        _, predicted = torch.max(outputs, 1)
        print(f"Predicted IDs: {predicted.cpu().numpy()}")
        print(f"Actual IDs:    {labels.numpy()}")
            
    print(f"Epoch [{epoch+1}/{num_epochs}] | Loss: {running_loss/len(train_loader):.4f} | Val Accuracy: {100 * correct / total:.2f}%")

print("🎉 Training Complete!")

Predicted IDs: [23 21  8  8 22 23 23 22 23 23 23 22  8  8  8 21 21 21 21 22  8  8  3  5
  3 23  5 23  3  3  3 23]
Actual IDs:    [0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 2 2 2 2 2 3 3 3 3 3 3 3 3 3 3 3 3]
Epoch [1/10] | Loss: 1.6523 | Val Accuracy: 72.22%
Predicted IDs: [ 0  0 13 14  0  0  0  0  0  0  0  0  1 14  1 21 10 21 21 22  3  3  3  3
  3  3  3  3  3  9  3  3]
Actual IDs:    [0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 2 2 2 2 2 3 3 3 3 3 3 3 3 3 3 3 3]
Epoch [2/10] | Loss: 0.7807 | Val Accuracy: 67.78%
Predicted IDs: [ 0  0 21 22  0  0  0  0  0  0  0  0  1  1  1 21 21 21  3  0  3  3  3  3
  3  3  3  3  3  3  3  3]
Actual IDs:    [0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 2 2 2 2 2 3 3 3 3 3 3 3 3 3 3 3 3]
Epoch [3/10] | Loss: 0.5201 | Val Accuracy: 88.69%
Predicted IDs: [ 0  0 21 22  0  0  0  0 23  0  0  0  1  1  1 16 21 21  3 22  3  3  3  3
  3  3  3  3  3  3  3  3]
Actual IDs:    [0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 2 2 2 2 2 3 3 3 3 3 3 3 3 3 3 3 3]
Epoch [4/10] | Loss: 0.3218 | Val Accuracy: 89.09%
Predicted IDs: [

In [9]:
# Define your save path
MODEL_PATH = "mars_model_94acc.pth"

# Save the model
torch.save({
    'model_state_dict': model.state_dict(),
    'num_classes': 24,
    'accuracy': 94.0
}, MODEL_PATH)

print(f"✅ Model saved to {MODEL_PATH}")

✅ Model saved to mars_model_94acc.pth
